In [1]:
#!python -m pip install --user --upgrade pip

     |████████████████████████████████| 1.5MB 7.5MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
#!pip3 install kfp --upgrade --user

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 170 kB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 3.4 MB/s  eta 0:00:01
     |████████████████████████████████| 54 kB 4.5 MB/s  eta 0:00:01
     |████████████████████████████████| 82 kB 1.3 MB/s  eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for kfp: filename=kfp-1.3.0-py3-none-any.whl size=235905 sha256=79bbd574e0eb51cdf86ec0a34172d34e098d351f15cfd547a6685116ef5fa91f
  Stored in directory: /home/jovyan/.cache/pip/wheels/56/f2/6d/72f2549e11209dc961adbfb71872b7d475f68ca97a80d2fea2
  Created wheel for docstring-parser: filename=docstring_parser-0.7.3-py3-none-any.whl size=19230 sha256=bbd8950acb2358a9701777ee1f035c4f5bbab1c67bf04585e0e47

### after installing the necessary packages, please restart kernel before continuing

###  importing some Kubeflow Pipeline SDK packages

In [1]:
import kfp
from kfp import dsl
import kfp.components as comp

### component that obtains data from source

In [2]:
def obtain_data_op():
    return dsl.ContainerOp(
        name = 'Obtain Data',
        image = 'mavencodev/obtain-data-component:v.0.1',
        arguments = [],
        file_outputs={
            'data': '/obtain_data/data'
        }      
    )

### component that handles preprocessing of the data from the obtain_data component

In [3]:
def preprocess_op(data):
    return dsl.ContainerOp(
        name = 'Preprocess Data',
        image = 'mavencodev/preprocess-component:v.0.1',
        arguments = ['--data', data],
        file_outputs={
            'X_train': '/preprocess_data/X_train.npy',
            'X_test': '/preprocess_data/X_test.npy',
            'y_train': '/preprocess_data/y_train.npy',
            'y_test': '/preprocess_data/y_test.npy'     
        }
    )

### component that handles training of the model using data from preprocess component

In [4]:
def train_op(X_train, y_train):
    return dsl.ContainerOp(
        name = 'Train Model',
        image = 'mavencodev/train-tensorflow:v.0.1' ,
        arguments = [
            '--X_train', X_train,
            '--y_train', y_train   
        ],
        file_outputs={
            'model':'/tf_data/classifier.h5'
        }
    )

### component that handles predicting of data using data from preprocess component and model from train component

In [5]:
def predict_op(X_test, y_test, model):
    return dsl.ContainerOp(
        name = 'Predictions',
        image = 'mavencodev/predict-tensorflow:v.0.1' ,
        arguments = [
            '--X_test', X_test,
            '--y_test', y_test,
            '--model', model    
        ],
        file_outputs={
            'performance':'/predict_data/performance.txt',
            'results':'/predict_data/results.txt'
        }
    )

### Defining the pipeline and how data is passed between components

In [6]:
@dsl.pipeline(
    name='Churn modelling pipeline',
   description='An ML reusable pipeline that performs customer segmentation to determine customers with high risk of leaving a bank .'
)

# Define parameters to be fed into pipeline
def churn_reuseable_tensorflow_pipeline(): 
    
        _obtain_data_op = obtain_data_op()

        _preprocess_op = preprocess_op(
            dsl.InputArgumentPath(_obtain_data_op.outputs['data'])).after(_obtain_data_op)

        _train_op = train_op(
            dsl.InputArgumentPath(_preprocess_op.outputs['X_train']),
            dsl.InputArgumentPath(_preprocess_op.outputs['y_train'])).after(_preprocess_op)

        _predict_op = predict_op(
            dsl.InputArgumentPath(_preprocess_op.outputs['X_test']),
            dsl.InputArgumentPath(_preprocess_op.outputs['y_test']),
            dsl.InputArgumentPath(_train_op.outputs['model'])).after(_train_op)

### compiling the pipeline

In [7]:
# Compile pipeline to generate compressed YAML definition of the pipeline.
experiment_name = 'churn_analysis_tensorflow_pipeline'
#kfp.compiler.Compiler().compile(churn_reuseable_tensorflow_pipeline,  
#  '{}.zip'.format(experiment_name))
kfp.compiler.Compiler().compile(churn_reuseable_tensorflow_pipeline, '{}.yaml'.format(experiment_name))

/home/jovyan/.local/lib/python3.6/site-packages/kfp/dsl/_container_op.py:1039: FutureWarning: Please create reusable components instead of constructing ContainerOp instances directly. Reusable components are shareable, portable and have compatibility and support guarantees. Please see the documentation: https://www.kubeflow.org/docs/pipelines/sdk/component-development/#writing-your-component-definition-file The components can be created manually (or, in case of python, using kfp.components.create_component_from_func or func_to_container_op) and then loaded using kfp.components.load_component_from_file, load_component_from_uri or load_component_from_text: https://kubeflow-pipelines.readthedocs.io/en/stable/source/kfp.components.html#kfp.components.load_component_from_file
  category=FutureWarning,


### running the pipeline

In [8]:
client = kfp.Client()
client.create_run_from_pipeline_func(churn_reuseable_tensorflow_pipeline, arguments={})

RunPipelineResult(run_id=ccaa1514-fe65-410b-973a-2e7386340e45)